https://cryptosalamander.tistory.com/141?category=1218889  
https://colab.research.google.com/drive/1lYBYtaXqt9S733OXdXvrvC09ysKFN30W#scrollTo=6S0xPged3n8S  
https://huggingface.co/blog/how-to-train  

--model_name_or_path : 기존 모델의 weight를 불러온 뒤, 전이학습을 할 때 사용된다.  
--model_type : pretrained weight 없이 구조만 가져와서 처음부터 학습을 진행할 시 사용한다.   
--tokenizer_name: pretrained tokenizer를 불러오거나, tokenizer가 저장된 폴더의 경로를 전달하여 tokenizer를 불러온다. 기본적으로, tokenizer_config.json, vocab.txt, special_tokens_map.json을 불러오며, 이는 이전 포스팅에서 학습한 tokenizer를 저장한 경로에 존재한다.  
--max_seq_length : input으로 사용될 문장의 길이 한도를 의미한다.  
--train_file : 학습에 사용될 Corpus 데이터를 전달하면 된다.   
--validation_file : validation 과정에 사용될 Corpus 데이터 경로를 전달하면 된다.  
--validation_split_percentage : 별도의 validation_file이 없을 경우, train set에서 몇퍼센트를 추출하여 validation_set으로 활용한다. 10정도 입력하면 적절하다.  
--cache_dir : 전처리된 임시 파일, 로그, checkpoint등이 저장될 경로, 입력하지 않으면 기본값인 nltk_data cache로 가지만, 지정해주면 해당 경로에 저장된다. 생각보다 용량이 커질 수 있으므로 용량이 넉넉한 파티션에 지정하는 것이 좋다.  
--preprocessing_num_workers : 전처리에 사용할 프로세스의 개수이다. CPU Core수에 맞게 적절히 세팅해주면 좋다. 기본값으로 돌릴 시 47시간 걸리던 전처리가, num_workers를 16으로 해줬더니 10시간으로 줄어들었다. 하지만 너무 클 경우 CPU와 메모리에 부하가 크므로, CPU 사양을 고려하여 설정해야한다.  
--line_by_line : 문장이 라인 단위로 쪼개져있는 Corpus에서 사용한다. 기본적으로 False 상태로 되어있다. 기본적으로 huggingface에서는 line_by_line이 꺼져있으면, 문장들을 합친 뒤 512를 넘기지 않도록 적절히 chunks를 만들고 이를 이용해 학습을 진행한다. line_by_line은 왠만하면 키지 않는 것이 좋다. 오용될 경우, Corpus에 불필요한 [PAD]가 매우 많이 발생하기 때문이다.  
--pad_to_max_length : 512보다 짧은 문장에 padding을 추가하여 512로 만들어준다. 기본적으로 켜주는게 좋지만, 잘못된 방식으로 line_by_line과 함께 사용될 경우, 전체 데이터 단어 1개에 [PAD] 토큰 99개가 붙는 최악의 상황이 발생할 수 있다.   
--per_device_train_batch_size : 말 그대로 train batch size  
--per_device_eval_batch_size : eval batch size  
--save_steps : 몇 step 마다 저장할 것인지, 기본값은 5000이지만, Corpus가 크다면 넉넉히 설정하는게 좋다. 너무 자주 저장되면 checkpoints 용량도 어마어마하게 크기 때문이다.

$python ./examples/pytorch/language-modeling/run_mlm.py \
  --max_seq_length 512 \
  --pad_to_max_length \
  --logging_dir ./tensorboard-metrics \
  --cache_dir /mnt/disks/sdc/cache_dir \
  --train_file ./train.txt \
  --tokenizer_name ./mytokenizer/ \
  --preprocessing_num_workers 8 \
  --do_train \
  --do_eval \
  --model_type bert \
  --validation_split_percentage 10 \
  --overwrite_output_dir \
  --output_dir my-bert \
  --num_train_epochs 3 \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --save_steps 500000

python ./transformers/examples/pytorch/language-modeling/run_mlm.py  --max_seq_length 512  --pad_to_max_length  --logging_dir ./logging  --cache_dir ./cache  --train_file data\open_research\sample_sents.txt  --tokenizer_name ./output/pretrained_tokenizer  --do_train  --do_eval  --model_type bart  --validation_split_percentage 10  --overwrite_output_dir  --output_dir ./output/bart  --num_train_epochs 3  --per_gpu_train_batch_size 4  --per_gpu_eval_batch_size 1  --save_steps 1000

In [ ]:
python ./examples/pytorch/language-modeling/run_mlm.py --max_seq_length 512 \
  --pad_to_max_length \
  --logging_dir ./tensorboard-metrics \
  --cache_dir /mnt/disks/sdc/cache_dir \
  --train_file ./train.txt \
  --tokenizer_name ./mytokenizer/ \
  --preprocessing_num_workers 8 \
  --do_train \
  --do_eval \
  --model_type bert \
  --validation_split_percentage 10 \
  --overwrite_output_dir \
  --output_dir my-bert \
  --num_train_epochs 3 \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --save_steps 500000

In [3]:
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
from tokenizers import ByteLevelBPETokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('%d GPU(s)' %torch.cuda.device_count())
else:
    print('CPU')
    device = torch.device('cpu')

1 GPU(s)


In [66]:
# corpus dataset 가져옴(torch Dataset 상속)
class CorpusDataset(Dataset):
    def __init__(self, txt:str='sample_sents.txt'):
        tokenizer = ByteLevelBPETokenizer(
            './output/pretrained_tokenizer/vocab.json',
            './output/pretrained_tokenizer/merges.txt'
        )  
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        
        self.data = []
        with open('data/open_research/'+txt, encoding='utf-8') as f:
            self.data = f.readlines()
            self.data = list(map(lambda x: x.strip(), data))
        self.data = [x.ids for x in tokenizer.encode_batch(self.data)]
    
    def __len__(self):      # 데이터셋 길이(샘플 수) return
        return len(self.data)
    
    def __getitem__(self, idx):      # idx번째 data(tensor) return
        return torch.tensor(self.data[idx])

In [67]:
dataset = CorpusDataset('sample_sents.txt')
dataset[0]

tensor([   2,  366,  279,  974, 2010, 1479,  287, 1456,  455,  348, 1456, 3584,
         582,   88,  628, 3838, 1308,    3])

In [61]:
# return mini batch
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)  # num_workers: 서버 개수
dataloader